In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats  
from fitter import Fitter, get_common_distributions, get_distributions

### sample initial conditions on Feb 7, 2021 for each run from log gamma distributions

We ran 250 independent runs initializing the model at the start of the epidemic in France (early 2020) up to February 2021. We then fitted a log gamma distribution on the prevalence in each compartment reached on Feb 7, 2021 (end of week 5). The parameters of the log gamma distributions were estimated at maximum likelihood. A log gamma distribution was selected as the best distribution in terms of AIC among a set of commonly used distributions.

To save computational time, we then re-initialized the model on Feb 7, 2021. For each run, the initial condition for each compartment was sampled from the corresponding log gamma distribution. This procedure allows to initialize the model in February 2021, mantaining the variability associated with the simulated trajectory seeded at the start of the epidemic and progressing throughout 2020.

In [ ]:
# input file with paramaters of log gammma distributions for each (non-null) compartment

day = '2021-02-07'
parameters = pd.read_csv('./input/parameters_loggamma_{}.csv'.format(day),sep=",")
parameters

In [ ]:
n_runs=250

# sample 250 different initial configurations

configuration=pd.DataFrame()

for comp in parameters.columns: 
    configuration[comp]=stats.loggamma.rvs(c=parameters[comp][0],loc=parameters[comp][1],scale=parameters[comp][2], size=250)
    configuration[comp]=configuration[comp].astype(int)

In [ ]:
# list of names of each compartment
col_names_tot = []

for age in range(4):
    col_names_tot.append('S_no_vax_{}'.format(age))
for age in range(4):
    col_names_tot.append('S_vax_{}'.format(age))

for strain in ['wild', 'VOC', 'wild_vax', 'VOC_vax']:
    for age in range(4):
        col_names_tot.append('E_{}_{}'.format(strain,age))
for strain in ['wild', 'VOC', 'wild_vax', 'VOC_vax']:
    for age in range(4):
        col_names_tot.append('I_p_{}_{}'.format(strain,age))
for strain in ['wild', 'VOC', 'wild_vax', 'VOC_vax']:
    for age in range(4):
        col_names_tot.append('I_as_{}_{}'.format(strain,age))
for strain in ['wild', 'VOC', 'wild_vax', 'VOC_vax']:
    for age in range(4):
        col_names_tot.append('I_ps_{}_{}'.format(strain,age))
for strain in ['wild', 'VOC', 'wild_vax', 'VOC_vax']:
    for age in range(4):
        col_names_tot.append('I_ms_{}_{}'.format(strain,age))
for strain in ['wild', 'VOC', 'wild_vax', 'VOC_vax']:
    for age in range(4):
        col_names_tot.append('I_ss_{}_{}'.format(strain,age))
for strain in ['wild', 'VOC', 'wild_vax', 'VOC_vax']:
    for age in range(4):
        col_names_tot.append('H_{}_{}'.format(strain,age))
for strain in ['wild', 'VOC', 'wild_vax', 'VOC_vax']:
    for age in range(4):
        col_names_tot.append('R_{}_{}'.format(strain,age))

In [ ]:
# add compartments with 0 individuals
for cc in col_names_tot:
    if cc not in configuration.columns:
        configuration[cc] = np.zeros(250)
    
configuration = configuration[col_names_tot]
configuration

In [ ]:
# population of IDF
N_c = 1590000   # young children
N_t = 1550000   # adolescents
N_a = 7250000   # adults
N_s = 1890000   # seniors

In [ ]:
# adjust number of individuals in S_no_vax so that sum of all compartments matches the target population
start_comp=8
jump=4

c2=configuration.iloc[0:,start_comp::jump]
configuration['S_no_vax_0']=int(N_c)-c2[configuration.iloc[0:,start_comp::jump].columns].sum(axis=1)

c2=configuration.iloc[0:,start_comp+1::jump]
configuration['S_no_vax_1']=int(N_t)-c2[configuration.iloc[0:,start_comp+1::jump].columns].sum(axis=1)

c2=configuration.iloc[0:,start_comp+2::jump]
configuration['S_no_vax_2']=int(N_a)-c2[configuration.iloc[0:,start_comp+2::jump].columns].sum(axis=1)

c2=configuration.iloc[0:,start_comp+3::jump]
configuration['S_no_vax_3']=int(N_s)-c2[configuration.iloc[0:,start_comp+3::jump].columns].sum(axis=1)

In [ ]:
#save results

configuration.to_csv('./input/configuration_loggamma_{}.csv'.format(day), index=False)